In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import sys
import numpy as np
import pandas as pd
import glob as glob
import scipy.stats
import pickle
import random as random
from sklearn import metrics as skm

from sklearn import preprocessing, model_selection


Mounted at /content/drive


In [2]:
# Requires Stellargraph (version 1.2.1)

import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN
from stellargraph import StellarGraph, datasets

from stellargraph.data import BiasedRandomWalk
from sklearn.manifold import TSNE

import networkx as nx

from gensim.models import Word2Vec

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None


     |████████████████████████████████| 435 kB 4.1 MB/s 
     |████████████████████████████████| 500 kB 38.7 MB/s 
     |████████████████████████████████| 120 kB 30.5 MB/s 
     |████████████████████████████████| 1.6 MB 43.9 MB/s 
     |████████████████████████████████| 82 kB 493 kB/s 
     |████████████████████████████████| 41 kB 526 kB/s 


In [5]:
# Whether to download results

save_results = False

In [ ]:
links = pd.read_csv('edges.csv')

human_ppi_nodes = list(set(list(links.gene1) + list(links.gene2)))

G = nx.Graph()
G.add_nodes_from(human_ppi_nodes)

positive_edges = list([x for x in zip(links.gene1,links.gene2)])
G.add_edges_from(positive_edges)


In [ ]:
G_sg = StellarGraph.from_networkx(G)

rw = BiasedRandomWalk(G_sg)

walks = rw.run(
    nodes=list(G_sg.nodes()),
    length = 100,
    n = 10,
    p = 0.5,
    q = 2.0,
)
print("Number of random walks: {}".format(len(walks)))

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=1)

node_ids = model.wv.index2word
node_embeddings = (
    model.wv.vectors
)

if save_results:
    pickle.dump(node_ids, open('humanppi_node_ids.p', 'wb'))
    pickle.dump(node_embeddings, open('humanppi_node_embs.p', 'wb'))
